In [26]:
from datetime import datetime

from forge_data import DataForge
from db_comm import PortfolioDBManager, DATABASE_NAME

In [27]:
pm = PortfolioDBManager(DATABASE_NAME)

Calculation Motor

In [28]:
from portfolio_test.motor import CalculationMotor

In [29]:
data_forge = DataForge()

In [30]:
cm = CalculationMotor("AAPL")

Build calculation motor for AAPL

In [80]:
pm.sell_all_assets()
starting_off_cash = 100
# withdraw cash
pm.record_transaction(
    tx_type="WITHDRAW", # WITHDRAW or DEPOSIT
    ticker="CASH",
    shares=pm.get_cash_balance(),
    actual_price=1,
    tx_datetime=datetime.now(),
)
pm.record_transaction(
    tx_type="DEPOSIT", # WITHDRAW or DEPOSIT
    ticker="CASH",
    shares=starting_off_cash,
    actual_price=1,
    tx_datetime=datetime.now(),
)

   -> Cash balance adjusted by 642.9595
2026-01-11 16:13:18: ✅ Recorded SELL: AAPL @ 642.9595. Snapshot updated.
2026-01-11 16:13:18: ✅ Recorded WITHDRAW: CASH @ 642.9595. Snapshot updated.
2026-01-11 16:13:18: ✅ Recorded DEPOSIT: CASH @ 100.0. Snapshot updated.


In [81]:
# if data exists
if "data" not in locals():
    data = data_forge.insert_data(cm.df["Adj Close"], column_name="Close (AAPL)")
data


,Close (AAPL),EMA_20,EMA_50,Signal
2020-01-02 00:00:00,673.954976,673.954976,673.954976,0
2020-01-03 00:00:00,667.402787,670.515077,670.613360,0
2020-01-06 00:00:00,672.720600,671.324932,671.344053,0
2020-01-07 00:00:00,669.556860,670.814517,670.870097,0
2020-01-08 00:00:00,680.327582,673.115646,672.915877,1
...,...,...,...,...
2026-01-05 00:00:00,2485.518091,2539.649011,2506.372102,1
2026-01-06 00:00:00,2439.947864,2530.153664,2503.767229,1
2026-01-07 00:00:00,2421.068875,2519.764637,2500.524157,1
2026-01-08 00:00:00,2409.072079,2509.222488,2496.937801,1


In [82]:
close_column = "Close (AAPL)"
data['EMA_20'] = data[close_column].ewm(span=20).mean().shift(0)
data['EMA_50'] = data[close_column].ewm(span=50).mean().shift(0)

data['Signal'] = 0
# Buy when 20 EMA > 50 EMA
data.loc[data['EMA_20'] > data['EMA_50'], 'Signal'] = 1
# Sell (or Cash) when 20 EMA < 50 EMA
data.loc[data['EMA_20'] < data['EMA_50'], 'Signal'] = 0


In [83]:
print(data.columns)
data

Index(['Close (AAPL)', 'EMA_20', 'EMA_50', 'Signal'], dtype='object')


,Close (AAPL),EMA_20,EMA_50,Signal
2020-01-02 00:00:00,673.954976,673.954976,673.954976,0
2020-01-03 00:00:00,667.402787,670.515077,670.613360,0
2020-01-06 00:00:00,672.720600,671.324932,671.344053,0
2020-01-07 00:00:00,669.556860,670.814517,670.870097,0
2020-01-08 00:00:00,680.327582,673.115646,672.915877,1
...,...,...,...,...
2026-01-05 00:00:00,2485.518091,2539.649011,2506.372102,1
2026-01-06 00:00:00,2439.947864,2530.153664,2503.767229,1
2026-01-07 00:00:00,2421.068875,2519.764637,2500.524157,1
2026-01-08 00:00:00,2409.072079,2509.222488,2496.937801,1


In [84]:
import numpy as np

for date in data.index:
    print(f"- - - - - Date: {date.date()} - - - - -")
    buy_signal =  data.loc[date, 'Signal']
    buy_signal_ydy = data['Signal'].shift(1).loc[date]

    price = data.loc[date, close_column]  # price in SEK

    if buy_signal and not buy_signal_ydy:
        cash_balance = pm.get_cash_balance()
        shares_to_buy = np.divide(cash_balance, price)
        if shares_to_buy > 0:
            print("Buying AAPL")
            pm.record_transaction(
                tx_type="BUY",
                ticker="AAPL",
                shares=shares_to_buy,
                actual_price=price,
                tx_datetime=date,
                currency="SEK",
            )
    elif not buy_signal and buy_signal_ydy:
        portfolio = pm.get_portfolio_snapshot()
        if "AAPL" in list(portfolio["ticker"]):
            shares_to_sell = portfolio[portfolio["ticker"] == "AAPL"]["net_shares"].iloc[0]
            if shares_to_sell > 0:
                pm.record_transaction(
                    tx_type="SELL",
                    ticker="AAPL",
                    shares=shares_to_sell,
                    actual_price=price,
                    tx_datetime=date,
                    currency="SEK",
                )


- - - - - Date: 2020-01-02 - - - - -
- - - - - Date: 2020-01-03 - - - - -
- - - - - Date: 2020-01-06 - - - - -
- - - - - Date: 2020-01-07 - - - - -
- - - - - Date: 2020-01-08 - - - - -
Buying AAPL
   -> Cash balance adjusted by -100.0
2020-01-08 00:00:00: ✅ Recorded BUY: AAPL @ 100.0. Snapshot updated.
- - - - - Date: 2020-01-09 - - - - -
- - - - - Date: 2020-01-10 - - - - -
- - - - - Date: 2020-01-13 - - - - -
- - - - - Date: 2020-01-14 - - - - -
- - - - - Date: 2020-01-15 - - - - -
- - - - - Date: 2020-01-16 - - - - -
- - - - - Date: 2020-01-17 - - - - -
- - - - - Date: 2020-01-21 - - - - -
- - - - - Date: 2020-01-22 - - - - -
- - - - - Date: 2020-01-23 - - - - -
- - - - - Date: 2020-01-24 - - - - -
- - - - - Date: 2020-01-27 - - - - -
- - - - - Date: 2020-01-28 - - - - -
- - - - - Date: 2020-01-29 - - - - -
- - - - - Date: 2020-01-30 - - - - -
- - - - - Date: 2020-01-31 - - - - -
- - - - - Date: 2020-02-03 - - - - -
- - - - - Date: 2020-02-04 - - - - -
- - - - - Date: 2020-02-05 - -

In [85]:
pm.get_portfolio_snapshot()

,ticker,net_shares,last_trade_price,total_position_value
0,AAPL,0.107147,1959.476482,209.9517


In [86]:
from portfolio_service import PortfolioService

ps = PortfolioService(pm, data)
earnings = ps.get_total_valuation()
trading_gain = earnings/starting_off_cash
print(f"Total portfolio value: {earnings} SEK")
print(f"Trading gain: {trading_gain:.3f} times")

Latest price for AAPL from column Close (AAPL) is 2412.140954589844
Total portfolio value: 258.4532648408674 SEK
Trading gain: 2.585 times


In [87]:
# Compare to buy and hold
last_price = data.loc[data.index.max(), close_column]
first_price = data.loc[data.index.min(), close_column]
holding_gain = last_price/first_price
print(f"Holding_gain: {holding_gain:.3f} times")

Holding_gain: 3.579 times


In [88]:
print(f"Last price: {(holding_gain-1)*100:.2f} %, Earnings: {(trading_gain-1)*100:.2f} %")
if trading_gain < holding_gain:
    print("Overall worse than buy and hold. "
          f"Specifically, just buying would have given you {((holding_gain-1)/(trading_gain-1)):.2f} times better results.")
else:
    print(f"Overall better than buy and hold")
    if holding_gain-1 > 0:
        print(f"{((trading_gain-1)/(holding_gain-1)):.2f} times better than buy and hold")
    elif trading_gain-1 > 0:
        print("Holding was a loss, but trading made a gain")
    else:
        print("Both trading and holding were a loss but trading lost less")

Last price: 257.91 %, Earnings: 158.45 %
Overall worse than buy and hold. Specifically, just buying would have given you 1.63 times better results.
